In [12]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot
from plotly.offline import iplot as iplot_offline
from plotly.plotly import iplot as iplot
from plotly.graph_objs import Scatter
init_notebook_mode() # run at the start of every ipython notebook to use plotly.offline
                     # this injects the plotly.js source files into the notebook

%matplotlib inline
import matplotlib as plt
plt.style.use('ggplot')
import pandas as pd
import cufflinks as cf
import numpy as np
from datetime import datetime
import plotly.plotly as py
import plotly.graph_objs as go

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [2]:
update_string = ' (updated January 30, 2018)'
# markets_csv = '/Users/coreypetty/PycharmProjects/TBP_metrics/raw_data/us-markets_2016-06-25.csv'
world_csv = '/Users/coreypetty/Dropbox/Projects/PycharmProjects/TBP_metrics/raw_data/2018-01-28_export_11344.csv'
states_csv = '/Users/coreypetty/Dropbox/Projects/PycharmProjects/TBP_metrics/raw_data/2018-01-26_export_11467.csv'

In [14]:
states = pd.DataFrame.from_csv(states_csv)
states['perc'] = states.requests / states.requests.sum() * 100
states.reset_index(inplace=True)
states['grouping'] = np.where(states.perc <= 2, 'other', states.region)

states.groupby('grouping').sum().reset_index().iplot(kind='pie', labels='grouping', values='requests', 
                                                      filename='TBP_plots/US-states-pie',
                                                      title='US States Download Breakdown'
                                                     )

In [8]:
world = pd.DataFrame.from_csv(world_csv)
world['perc'] = world.requests / world.requests.sum() * 100
world.reset_index(inplace=True)
world['grouping'] = np.where(world.perc <= 2, 'other', world.region)

world.groupby('grouping').sum().reset_index().iplot(kind='pie', labels='grouping', values='total_downloads', 
                                                      filename='TBP_plots/world-pie',
                                                      title='Country Download Breakdown'
                                                     )

AttributeError: 'DataFrame' object has no attribute 'iplot'

In [4]:
world = pd.DataFrame.from_csv(world_csv)
world['perc'] = world.total_downloads / world.total_downloads.sum() * 100
world.reset_index(inplace=True)
world['grouping'] = np.where(world.perc <= 2, 'other', world.country_name)

world.groupby('grouping').sum().reset_index().iplot(kind='pie', labels='grouping', values='total_downloads', 
                                                      filename='TBP_plots/world-pie',
                                                      title='Country Download Breakdown'
                                                     )

AttributeError: 'DataFrame' object has no attribute 'total_downloads'

In [15]:
from scripts.geo_dicts import states as states_dict
from scripts.geo_dicts import countries as countries_dict
states_dict = dict((k,v) for v, k in states_dict.items())
countries_dict = dict((k,v) for v, k in countries_dict.items())
def get_abbrev(state_text):
    state = state_text.split(',')[0]
    if state in states_dict.keys():
        return states_dict[state]
def get_state_name(state_text):
    return state_text.split(',')[0]
def get_abbrev_country(name):
    if name in countries_dict.keys():
        return countries_dict[name]

In [17]:
import pycountry

countries = {}
for country in pycountry.countries:
    countries[country.name] = country.alpha_3
countries['United States'] = 'None'
codes = [countries.get(country, 'None') for country in world.region]

In [19]:
world[world['region'] == "United States"].index[0]

121

In [20]:
world.set_value(col='requests', index=world[world['region'] == "United States"].index[0], value=0)

,region,requests,perc,grouping
0,Anonymous Proxy,12,0.039556,other
1,United Arab Emirates,83,0.273593,other
2,Afghanistan,1,0.003296,other
3,Antigua and Barbuda,1,0.003296,other
4,Albania,6,0.019778,other
5,Armenia,3,0.009889,other
6,Argentina,132,0.435112,other
7,Austria,129,0.425223,other
8,Australia,1493,4.921383,Australia
9,Aland Islands,5,0.016482,other


In [21]:
scl = [[0.0, '#f1f1f1'],[0.2, '#efe7de'],[0.4, '#ecceaf'],\
            [0.6, '#ecb983'],[0.8, '#eca559'],[1.0, '#ef9534']]

us_data = go.Choropleth(
#     colorscale = scl,
    name = 'US States',
    autocolorscale = True,
    locations = states.region.apply(get_abbrev),
    z = states.requests,
    locationmode = 'USA-states',
    text = states.region.apply(get_state_name),
    marker = dict(
        line = dict (
            width = 0.5
        ) ),
    colorbar = dict(
        title = "USA")
)
world_data = go.Choropleth(
    colorscale = scl,
    name = 'World',
    autocolorscale = True,
    locations = codes,
    z = world.requests.astype('float'),
    text = world.region,
    locationmode = 'countries',
    marker = dict(
        line = dict (
            width = 1
        ) ),
    colorbar = dict(
        title = "World",
        x = 0.9,
    )
)
data = [us_data, world_data]
layout = dict(
    title = 'Downloads by Location' + update_string,
    geo = dict(
        scope='world',
        projection=dict( type='orthographic', ),
        showlakes = True,
        showocean = True,
        lakecolor = 'rgba(0, 204, 255, 1)',
        oceancolor = 'rgba(0, 204, 255, 0.5)',
        bgcolor = 'rgba(250,250,250,0)',
    ),
    
    paper_bgcolor='rgba(250,250,250,0)',
)
fig=dict(data=data, layout=layout)
iplot(fig, filename='TBP_plots/geo_choropleth')